In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn import svm
from sklearn.metrics import f1_score
#from sklearn.metrics import score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#from ipykernel import kernelapp as app
%matplotlib inline

In [57]:
data = pd.read_csv('Training Dataset.csv')
data.head()

,SFH,popup,ssl_fs,rqst_url,url_of_anchor,web_trffc,url_length,age_of_domain,ip address,Result
0,1,-1,1,-1,-1,1,1,1,0,0
1,-1,-1,-1,-1,-1,0,1,1,1,1
2,1,-1,0,0,-1,0,-1,1,0,1
3,1,0,1,-1,-1,0,1,1,0,0
4,-1,-1,1,-1,0,0,-1,1,0,1


In [58]:
###################filtered out phishing instances.################################# ---> DONE

phish = data[data.Result ==1]
leg = data[data.Result ==-1]
susp = data[data.Result == 0]

In [59]:
#fileterd out values in each column to figure out value with least number of occurence ---> counting done
#histogram for each features ---> To be done



###############finding the occurences of each value in each column############################## -->DONE
columns = list(phish)
columns.remove('Result')


melted_data_phish = pd.melt(phish, value_vars=columns, var_name='column', value_name='values')
count_phish = melted_data_phish.groupby(by=['column', 'values'])['values'].count()



melted_data_leg = pd.melt(leg, value_vars=columns, var_name='column', value_name='values')
count_leg = melted_data_leg.groupby(by=['column', 'values'])['values'].count()

print(count_phish)
print(count_leg)

column         values
SFH            -1        340
                0        125
                1         83
age_of_domain  -1        293
                1        255
ip address      0        498
                1         50
popup          -1        362
                0        186
rqst_url       -1        284
                0        212
                1         52
ssl_fs         -1        236
                0        181
                1        131
url_length     -1        229
                0        230
                1         89
url_of_anchor  -1        335
                0         51
                1        162
web_trffc      -1         98
                0        216
                1        234
Name: values, dtype: int64
column         values
SFH            -1         59
                0          2
                1        641
age_of_domain  -1        207
                1        495
ip address      0        610
                1         92
popup          -1        122
 

In [60]:
#replace the values in phishing instances with those found above and check the prediction

################################replaced two features with 1 value################################## -->Done
phish['SFH'] = phish['SFH'].replace([1, 0, -1], 1)
phish['popup'] = phish['popup'].replace([1, 0, -1], 1)



/home/achu/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/achu/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [61]:

########Appending the modifies phish data with unmodifeid legitimate data in dataset_final.csv###################

phish.to_csv('replaced.csv')
with open('replaced.csv', 'a') as f:
    leg.to_csv(f, header=False)
    susp.to_csv(f, header = False)


In [62]:
X_test = pd.read_csv('replaced.csv')
X_test = X_test.drop(['Result','Unnamed: 0'], axis = 1)
X_test.head()

,SFH,popup,ssl_fs,rqst_url,url_of_anchor,web_trffc,url_length,age_of_domain,ip address
0,1,1,-1,-1,-1,0,1,1,1
1,1,1,0,0,-1,0,-1,1,0
2,1,1,1,-1,0,0,-1,1,0
3,1,1,1,-1,-1,1,0,-1,0
4,1,1,-1,-1,1,1,0,-1,0


In [63]:
X = data.drop('Result',axis=1).values
y = data['Result'].values

classifier = svm.SVC(gamma='auto', kernel='rbf',degree =2)
classifier.fit(X,y)
accuracy_scores_2 = 100*cross_val_score(classifier, X, y, cv=5)
print(np.mean(accuracy_scores_2))

85.59378689129116


In [64]:
prediction_with_mod = classifier.predict(X_test)


In [65]:

print("Accuracy with same train set with 2 feature modification is: ",100*(accuracy_score(y,prediction_with_mod)))

Accuracy with same train set with 2 feature modification is:  49.667405764966745


In [ ]:
# print(predictions)
# np.savetxt("predict.csv", predictions, delimiter=",")